# Launch 12 Data First Look

Launch 12 lifted off at 11:15 am Sunday July 19th (19:15 UTC)

How did we do? Let's take a first look at the data:

In [ ]:
import numpy
import utils
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')

## Launch Time

Look for the umbilical disconnect signal. We'll write that down at $t_0$

In [ ]:
with open('../fc-data/RNHU.csv', 'r') as rnhu:
    for line in rnhu:
        print line,

t_0 = 117853569585227

## Acceleration

Here are the acceleration curves for the flight.

In [ ]:
columns = numpy.loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
gyro_x = columns[3]
gyro_y = columns[4]
gyro_z = columns[5]
acc_x = columns[6]
acc_y = columns[7]
acc_z = columns[8]

timestamp = numpy.subtract(timestamp, t_0)
timestamp = numpy.divide(timestamp, 1e9)

# Peak Thrust
thrust_peak = numpy.argmax(acc_x[0:25000])
thrust_peak = (timestamp[thrust_peak], acc_x[thrust_peak])

# Peak Decel
decel_peak = numpy.argmin(acc_x[0:25000])
decel_peak = (timestamp[decel_peak], acc_x[decel_peak])

# NSR Fire
nsr_event = numpy.argmax(acc_x[27100:29000])
nsr_event = (timestamp[27100+nsr_event], acc_x[27100+nsr_event])

fig = utils.Plot("Launch 12 Measured Vertical Vcceleration", "Time [s]", "Accel [m/s/s]")
fig.plot(timestamp, acc_x, label="UP Measured Acceleration")
fig.xlim([-2, 40])
fig.ylim([-50, 150])

fig.note('Peak Thrust (%0.1f m/s/s)' % thrust_peak[1], (thrust_peak[0], thrust_peak[1]), (50,20))
fig.note('Event?? (T+%0.3f)' % nsr_event[0], (nsr_event[0], nsr_event[1]), (1, 160))
fig.show()

In [ ]:
fig = utils.Plot("Launch 12 Measured Acceleration", "Time [sec]", "Accel [m/s/s]")
fig.plot(timestamp, acc_x, color=utils.red, label="UP Measured Acceleration")
fig.plot(timestamp, acc_y, color=utils.green, label="'Y' Measured Acceleration")
fig.plot(timestamp, acc_z, color=utils.blue, label="'Z' Measured Acceleration")
fig.legend()
fig.xlim([-2, 40])
fig.show()

# Roll Rate

This launch included a student designed roll control experiment. Four small canards in the middle of the rocket are controled by the flight comptuer and a PI controller was allowed to control the roll rate of the rocket, trying to set it to zero.

In [ ]:
fig = utils.Plot("Launch 12 Roll Rate", "Time [s]", r"Roll Rate [${}^0$/s]")
fig.plot(timestamp, gyro_x)
fig.xlim([-5, 40])
fig.ylim([-100, 150])

# Total Roll Angle

How much did we spin?

In [ ]:
from scipy.integrate import simps

roll_angle = [0]
for i in range(1, len(timestamp)):
    roll_angle.append(simps(gyro_x[:i], timestamp[:i]))

In [ ]:
fig = utils.Plot("Launch 12 Roll Angle", "Time [s]", r"Roll Rate [${}^0$]")
fig.plot(timestamp, roll_angle)
fig.xlim([-5, 40])
fig.show()

This was worse that expected performance, but also includes a lot of data that we'll find useful for future flights!

## Roll Canard Angle

So how did the control system react to the flight?

In [ ]:
columns = numpy.loadtxt("../fc-data/ROLL.csv", delimiter=',', unpack=True)

seqn = columns[0]
fin_timestamp = columns[1]
fin_angle = columns[2]

fin_timestamp /= 1e9
fin_timestamp -= 3348.1

fig = utils.Plot("Launch 12 Control Fin Angle", "Time [s]", r"Fin Angle [${}^0$]")
fig.plot(fin_timestamp, fin_angle)
fig.xlim([-2, 40])
fig.ylim([-5, 16])
fig.show()

In [ ]:
fig = utils.Plot("Launch 12 Control Fin Angle vs Roll Rate", "Time [s]", r"Fin Angle [${}^0/10$] Roll Rate [${}^0/s$]")
fig.plot(fin_timestamp, fin_angle*10, label="Fin Position")
fig.plot(timestamp, gyro_x, color=utils.blue, label="Roll Rate")
fig.xlim([-1, 32])
fig.ylim([-50, 160])
fig.legend(4)
fig.show()